In [1]:
%load_ext autoreload
%autoreload 1
%aimport Utils
%aimport MatrixLinkGenerator
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))
# np.set_printoptions(linewidth=np.inf)
from obspy import UTCDateTime as dt
import json
import matplotlib.pyplot as plt
from Utils import trainingResults, trainingResults2, predsMap
# plt.rcParams['figure.figsize'] = [50, 200]
plt.rcParams['figure.figsize'] = [16, 12]
params = json.loads('''{
    "extents": {
        "ak": {
            "latMin": 55.0,
            "latMax": 74.0,
            "lonMin": -163.0,
            "lonMax": -130.0
        },
        "s1": {
            "latMin": 22.0,
            "latMax": 40.0,
            "lonMin": 33.0,
            "lonMax": 62.0
        },
        "global": {
            "latMin": -90.0,
            "latMax": 90.0,
            "lonMin": -180.0,
            "lonMax": 180.0
        }
    },
    "location": "global",
    "maxDepth": 50.0,
    "maxStationElevation": 1.0,
    "trainingGeneratorSourceFile": "./Inputs/IDC 10-20.gz",
    "trainingEventsFile": "./Training/Event Files/IDC 10-20 ECEF Stations Times.npz",
    "validationGeneratorSourceFile": "./Inputs/IDC 10-20.gz",
    "validationEventsFile": "./Training/Event Files/IDC 10-20 ECEF Stations Times.npz",
    "arrivalProbsFile": "./Training/RSTT Model/S1 Dropouts.npy",
    "stationFile": "./Archive/Stations/S1 Station List.txt",
    "oneHot": "True",
    "arrivalProbMods": {
        "Pg": 5.0,
        "Pn": 3.0,
        "Sg": 5.0,
        "Sn": 25.0
    },
    "eventsPerExample": {
        "min": 6,
        "max": 20
    },
    "stationsPerBatch": {
        "min": 45,
        "max": 55
    },
    "timeShifts": {
        "min": -0.20,
        "max": 0.40
    },
    "batchSize": 500,
    "samplesPerEpoch": 1000000,
    "validationSamplesPerEpoch": 250000,
    "epochs": 1000,
    "model": "./Training/Models/IDC/E054 L0.0017 AL0.0009 LL172.8020 TL0.0002 AA0.1080 AP0.8677 AR0.7861.h5",
    "evalInFile": "./Inputs/S1 00.gz",
    "evalOutFile": "./Training/Evaluation.gz",
    "prlEvalOutFile": "./Training/PRL Evaluation.gz",
    "maxArrivals": 250,
    "minArrivals": 5,
    "maxNoise": 0.20,
    "clusterStrength": 0.9,
    "timeNormalize": 3600,
    "associationWindow": 3600,
    "evalWindow": 10.0,
    "phases": {
        "P": 0,
        "LR": 1,
        "Pn": 2,
        "T": 3,
        "tx": 4,
        "N": 5,
        "Sx": 6,
        "Pg": 7,
        "Lg": 8,
        "Sn": 9,
        "S": 10
    },
    "modelArch": {
        "dense": [32, 32, 32, 64, 64],
        "transformers": [128, 128],
        "heads": 4,
        "dense2": [128, 128, 128],
        "grus": [128, 128]
    }
}''')
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import random
import rstt
from copy import deepcopy
from collections import deque
modelPath = "./Training/RSTT Model/pdu202009Du.geotess"

def generateEventFile(params, trainingSet = False):
    if trainingSet:
        eventsFile = params['trainingEventsFile']
        generatorFile = params['trainingGeneratorSourceFile']
    else:
        eventsFile = params['validationEventsFile']
        generatorFile = params['validationGeneratorSourceFile']
    try:
        evFile = np.load(eventsFile, allow_pickle=True)
        events = evFile['events'].flatten()[0]
        eventsList = evFile['eventsList']
        print("Training events loaded.") if trainingSet else print("Validation events loaded.")
    except:
        print("Events not loaded. Building from scratch.")
        extents = np.array(list(params['extents'][params['location']].values())+[params['maxDepth'],params['maxStationElevation']])
        latRange = abs(extents[1] - extents[0])
        lonRange = abs(extents[3] - extents[2])
        timeNormalize = params['timeNormalize']
        phases = params['phases']
        events = {}
        eventsList = []
        inputArrivals = pd.read_pickle(generatorFile).sort_values(by="TIME")
        groupedEvents = (inputArrivals.groupby('EVID').filter(lambda x: len(x) >= params['minArrivals'])).groupby('EVID')
        count = 0
        for eid, arrivals in groupedEvents:
            count += 1
            print("\rBuilding event list: " + str(count) + ' / ' + str(len(groupedEvents)), end='')
            eventArrivals = []
            first = dt(arrivals.TIME.min())
            evtime = -(first - dt(arrivals.EV_TIME.min())) / timeNormalize
            for i, arrival in arrivals.iterrows():
                sx,sy,sz = ll2ecef(arrival.ST_LAT, arrival.ST_LON)
                thisArrival = [sx,                                        # normalized station x
                               sy,                                        # normalized station y
                               sz,                                        # normalized station z
                               ((dt(arrival.TIME)-first)/timeNormalize),  # normalized arrival time
                               phases[arrival.PHASE],                     # phase
                               1.,                                        # valid arrival flag
                               5.0 / timeNormalize,                       # arrival uncertainty
                               0.9,                                       # retention rate when dropping some arrivals
                               abs((arrival.EV_LAT-extents[0])/latRange), # normalized event lat
                               abs((arrival.EV_LON-extents[2])/lonRange), # normalized event lon
                               evtime]                                    # normalized event time (relative to first arrival)
                eventArrivals.append(thisArrival)
            events[eid] = np.array(eventArrivals)
            eventsList.append(np.array([eid,arrivals.TIME.min()]))
        eventsList = np.array(eventsList)
        eventsList = eventsList[np.argsort(eventsList[:,1])]
        np.savez_compressed(eventsFile, events=events, eventsList=eventsList)
        print()
    return events, eventsList

def buildAssociationMatrix(evids):
    L = np.zeros((len(evids), len(evids))) + 99
    sparse_evids = evids[evids>=0]
    l = np.ones((len(sparse_evids), len(sparse_evids))) * sparse_evids.reshape((-1, 1))
    L[:len(sparse_evids), :len(sparse_evids)] = (l == l.T) * 1
    return L

def synthesizeEventsFromEventFile(params, events, eventList, trainingSet = False):
    maxArrivals = params['maxArrivals']
    minTimeShift = params['timeShifts']['min']
    maxTimeShift = params['timeShifts']['max']
    minEvents = params['eventsPerExample']['min']
    maxEvents = params['eventsPerExample']['max']+1 # because using in np.random.randint
    dropFactor = 0.5
    batchSize = params['batchSize']
    start = eventList[0][1]
    end = eventList[-1][1]
    time = start

    while True:
        X = []
        Y = []
        example = 0
        while example < batchSize:
            #Setup - choose random events, with the first being the primary event
#             numEvents = np.random.randint(minEvents, maxEvents)
#             chosenEvents = random.sample(eventList, numEvents)

            chosenEvents = np.where((trainingEventList[:,1] >= time) & (trainingEventList[:,1] <= time+timeNormalize))[0]
            numEvents = len(chosenEvents)

#             timeShifts = np.random.uniform(minTimeShift, maxTimeShift, size=numEvents-1)
            for i in range(0, len(chosenEvents)):
                thisEvent = events[eventList[chosenEvents[i]][0]]
                #Randomly drop some picks from the event
                if trainingSet:
                    drops = thisEvent[:,7]
                    drops = drops + dropFactor*(1-drops) if dropFactor > 0 else drops*(1+dropFactor)
                    drops = np.random.binomial(1,drops)
                    idx = np.where(drops==1)[0]
                    thisEvent = thisEvent[idx,:]
                if i == 0:
                    sequence = thisEvent
                    sequence[:,5] = i
                else:
                    #Add the picks from this event
                    currentLength = len(sequence)
                    sequence = np.append(sequence, thisEvent, axis=0)
                    #Shift the starting time of this event
#                     sequence[currentLength:currentLength+len(thisEvent),[3,10]] += timeShifts[i-1]
                    sequence[currentLength:currentLength+len(thisEvent),[3,10]] += ((eventList[chosenEvents[i]][1] - time)/timeNormalize)
                    sequence[currentLength:currentLength+len(thisEvent),5] = i
            if time > end:
                time = start
            else:
                time += timeNormalize
            if numEvents == 0:
                continue
        
            #Add random arrival time errors, except for the first pick of the primary event
            if trainingSet:
                timeShifts = np.random.uniform(-sequence[1:,6]*0.5, sequence[1:,6]*0.5)
                sequence[1:,3] += timeShifts

            #Sort by arrival time, drop picks with negative arrival times
            idx = np.argsort(sequence[:,3])
            remove = len(np.where((sequence[:,3] < 0))[0])
            idx = idx[remove:]
            sequence = sequence[idx,:]
            if len(sequence) == 0: #We lost all the valid arrivals, so scrap this training example
                continue

            #Make labels array
            labels = buildAssociationMatrix(sequence[:,5])

            #Reset primary event times
            ones = np.where(labels[0]==1)
#             if len(ones[0]) == 0: #We lost all the valid arrivals, so scrap this training example
#                 continue
            sequence[ones,3] -= sequence[ones,3][0][0]
            idx = np.argsort(sequence[:,3])

            #Truncate picks over maximum allowed
            idx = idx[:maxArrivals]
            sequence = sequence[idx,:]
            labels = labels[idx,:maxArrivals]

            sequence[:,5] = 1.
            #Pad the end if not enough picks were selected
            padding = maxArrivals - len(sequence)
            if padding > 0:
                labels = np.pad(labels, (0, maxArrivals-len(labels)), constant_values=99.)
                sequence_ = np.full((maxArrivals, 11), 0.)
#                 sequence_[sequence.shape[0]:, [8,9]] = 99.
                sequence_[:sequence.shape[0], :] = sequence
                sequence = sequence_
            X.append(sequence)
            Y.append(labels)
            example += 1
            
        #Yield these training examples
        X = np.array(X)
        Y = {"association": np.array(Y), "location": X[:,:,[8,9]], "time": X[:,:,10]}
        X = {"phase": X[:,:,4], "numerical_features": X[:,:,[0,1,2,3,5]]}
        yield X, Y

In [3]:
maxArrivals = params['maxArrivals']
from tensorflow.keras import backend as K
from geographiclib.geodesic import Geodesic
from tensorflow.keras.losses import binary_crossentropy as BCE
from tensorflow.keras.metrics import binary_accuracy

K.set_floatx('float64')
maxArrivals = params['maxArrivals']
matrixSize = maxArrivals**2
extents = np.array(list(params['extents'][params['location']].values())+[params['maxDepth'],params['maxStationElevation']])
latRange = abs(extents[1] - extents[0])
lonRange = abs(extents[3] - extents[2])
timeNormalize = params['timeNormalize']
zero = np.float64(0)
one = np.float64(1)
r = np.float64(12756.2)
nn = np.float64(99)

import pymap3d as pm
xym = 6378137.0
xym2 = 2*xym
zm = 6356752.3142451802
zm2 = 2*zm
def ll2ecef(lat,lon):
    x,y,z = pm.geodetic2ecef(lat, lon, 0)
    x = (x+xym)/xym2
    y = (y+xym)/xym2
    z = (z+zm)/zm2
    return x,y,z
def ecef2ll(x,y,z):
    x = x*xym2 - xym
    y = y*xym2 - xym
    z = z*zm2 - zm
    x,y,z = pm.ecef2geodetic(x,y,z)
    return x,y

def nzHaversine(y_true, y_pred):
#     y_pred = y_pred * tf.cast(y_true != nn, tf.float64)
#     y_true = y_true * tf.cast(y_true != nn, tf.float64)
    observation = tf.stack([y_true[:,:,0]*latRange + extents[0], y_true[:,:,1]*lonRange + extents[2]],axis=2)*0.017453292519943295
    prediction = tf.stack([y_pred[:,:,0]*latRange + extents[0], y_pred[:,:,1]*lonRange + extents[2]],axis=2)*0.017453292519943295
    used = tf.reduce_sum(tf.cast(tf.greater(tf.reduce_sum(y_true, axis=2),0), dtype=tf.float64), axis=1)
    used = tf.where(tf.equal(used, zero), one, used)
    dlat_dlon = (observation - prediction) / 2
    a = tf.sin(dlat_dlon[:,:,0])**2 + tf.cos(observation[:,:,0]) * tf.cos(prediction[:,:,0]) * tf.sin(dlat_dlon[:,:,1])**2
    c = tf.asin(tf.sqrt(a))*r
    return tf.reduce_sum((tf.reduce_sum(c, axis=1))/used) / tf.dtypes.cast(tf.shape(observation)[0], dtype=tf.float64)

def nzTime(y_true, y_pred):
    y_pred = y_pred * tf.cast(y_true != nn, tf.float64)
    y_true = y_true * tf.cast(y_true != nn, tf.float64)
    used = maxArrivals - tf.reduce_sum(tf.cast(tf.equal(y_true, zero), dtype=tf.float64), axis=1)
    used = tf.where(tf.equal(used, zero), one, used)
    diffs = tf.math.abs(tf.squeeze(y_pred)-y_true)*timeNormalize
    diffs = (tf.squeeze(y_pred)-y_true)*timeNormalize
    diffs = tf.reduce_sum(tf.reduce_sum(diffs, axis=1)/used)
    return diffs/tf.dtypes.cast(tf.shape(y_true)[0], dtype= tf.float64)

def nzMSE(y_true, y_pred):
    y_pred = y_pred * tf.cast(y_true != nn, tf.float64)
    y_true = y_true * tf.cast(y_true != nn, tf.float64)
    used = maxArrivals - tf.reduce_sum(tf.cast(tf.equal(y_true,0), dtype=tf.float64), axis=1)
    used = tf.where(tf.equal(used, zero), one, used)
    return K.mean(tf.reduce_sum(K.square(tf.squeeze(y_pred)-y_true),axis=1)/used)

def nzBCE(y_true, y_pred):
    y_pred = y_pred * tf.cast(y_true != nn, tf.float64)
    y_true = y_true * tf.cast(y_true != nn, tf.float64)
    used = maxArrivals - tf.reduce_sum(tf.cast(tf.equal(y_true,0), dtype=tf.float64), axis=1)
    used = tf.where(tf.equal(used, zero), one, used)
    return K.mean(BCE(y_true, y_pred)/used)

def nzAccuracy(y_true, y_pred):
    y_pred = tf.squeeze(y_pred) * tf.cast(y_true != nn, tf.float64)
    y_true = y_true * tf.cast(y_true != nn, tf.float64)
    return K.mean(binary_accuracy(y_true, y_pred))

def nzRecall(y_true, y_pred):
    y_pred = y_pred * tf.cast(y_true != nn, tf.float64)
    y_true = y_true * tf.cast(y_true != nn, tf.float64)
#     y_true = K.ones_like(y_true)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    all_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (all_positives + K.epsilon())

def nzPrecision(y_true, y_pred):
    y_pred = y_pred * tf.cast(y_true != nn, tf.float64)
    y_true = y_true * tf.cast(y_true != nn, tf.float64)
#     y_true = K.ones_like(y_true)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return true_positives / (predicted_positives + K.epsilon())

In [4]:
#MatrixLinkTrainer
import tensorflow as tf
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Input, Embedding, Reshape, concatenate, Dense, Bidirectional, GRU, MultiHeadAttention, LayerNormalization, Lambda
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger
from tensorflow.keras.backend import clip
import logging
import json

@tf.autograph.experimental.do_not_convert
def MatrixLink(params):
    logging.getLogger("tensorflow").setLevel(logging.ERROR)
    def buildModel(modelArch):
        outputs = []
        inputs = []
        numericalInputs = Input(shape=(None,5), name='numerical_features')
        outputs.append(numericalInputs)
        inputs.append(numericalInputs)
        categoricalInputs = Input(shape=(None,1), name='phase')
        embed = Embedding(11, 4, trainable=True, embeddings_initializer=RandomNormal())(categoricalInputs)
        embed = Reshape(target_shape=(-1, 4))(embed)
        outputs.append(embed)
        inputs.append(categoricalInputs)
        outputs = concatenate(outputs)

        def TransformerBlock(inputs, embed_dim, ff_dim, num_heads=2, rate=0.1, eps=1e-6):
            attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
#             attn_output = Dropout(rate)(attn_output)
            out1 = LayerNormalization(epsilon=eps)(inputs + attn_output)
            ffn_output = Dense(ff_dim, activation="relu")(out1)
            ffn_output = Dense(embed_dim)(ffn_output)
#             ffn_output = Dropout(rate)(ffn_output)
            return LayerNormalization(epsilon=eps)(out1 + ffn_output) 

        for d1Units in modelArch['dense']:
            outputs = Dense(units=d1Units, activation=tf.nn.relu)(outputs)
        transformerOutputs = outputs
        gruOutputs = outputs

        for tUnits in modelArch['transformers']:
            transformerOutputs = TransformerBlock(transformerOutputs, d1Units, tUnits, modelArch['heads'])
        for gUnits in modelArch['grus']:
            gruOutputs = Bidirectional(GRU(gUnits, return_sequences=True))(gruOutputs)

        outputs = concatenate([transformerOutputs, gruOutputs], axis=2)
        for tUnits in modelArch['transformers']:
            outputs = TransformerBlock(outputs, d1Units+gUnits*2, tUnits, modelArch['heads'])

        association = Dense(units=params['maxArrivals'], activation=tf.nn.sigmoid, name='association')(outputs)
        location = Dense(units=2)(outputs)
        location = Lambda(lambda x: clip(x, 0, 1), name='location')(location)
        time = Dense(units=1, name='time')(outputs)
        
        model = Model(inputs=inputs, outputs=[association, location, time])
        losses = { 'association': nzBCE, 'location': nzHaversine, 'time': nzMSE }
        weights = { 'association': 1.0, 'location': 0.000004, 'time': 0.25 }
        metrics = { 'association': [nzAccuracy, nzPrecision, nzRecall] }
        model.compile(optimizer=Adam(clipnorm=0.00001), loss=losses, loss_weights=weights, metrics=metrics)
        return model

    model = buildModel(params['modelArch'])
    try:
        model.load_weights(params['model'])
        print("Loaded previous weights.")
    except Exception as e:
        print(e)
        print("No previous weights loaded.")
    print(model.summary())
    return model

class saveCb(Callback):
    def on_train_begin(self, logs=None):
        self.best = 100000000.
    def on_epoch_end(self, epoch, logs=None):
        if logs['loss'] < self.best:
            self.best = logs['loss']
            print('Saving best model with loss', self.best)
            modelName = 'E%03d L%.4f AL%.4f LL%.4f TL%.4f AA%.4f AP%.4f AR%.4f.h5' %\
                (epoch, logs['loss'], logs['association_loss'], logs['location_loss'], logs['time_loss'], logs['association_nzAccuracy'], logs['association_nzPrecision'], logs['association_nzRecall'])
            model.save("./Training/Models/IDC/"+modelName)

In [ ]:
# tf.config.threading.set_intra_op_parallelism_threads(2)
# tf.config.threading.set_inter_op_parallelism_threads(2)

trainingEvents, trainingEventList = generateEventFile(params, trainingSet=True)
for event in trainingEventList[:,0]:
    trainingEvents[event] = trainingEvents[event][np.where(trainingEvents[event][:,3] <= .6)[0]]
# validationEvents, validationEventList = generateEventFile(params)

generator = synthesizeEventsFromEventFile(params, trainingEvents, trainingEventList, trainingSet=False)
# generator = synthesizeEvents(params)
# vgen = synthesizeEventsFromEventFile(params, validationEvents, validationEventList)
# vgen = synthesizeEvents(params)

model = MatrixLink(params)
history = model.fit(generator,
#                  validation_data=vgen,
                 steps_per_epoch= params['samplesPerEpoch']/params['batchSize'],
#                  validation_steps = params['validationSamplesPerEpoch']/params['batchSize'],
                 epochs=params['epochs'],
                 callbacks=[saveCb(), EarlyStopping(monitor='loss', patience=50), CSVLogger('./Training/Models/IDC/logs.csv', append = True)],
                 verbose=1)
trainingResults(np.genfromtxt('./Training/Models/IDC/logs.csv', delimiter=',', names=True))

Training events loaded.
Loaded previous weights.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
phase (InputLayer)              [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 1, 4)   44          phase[0][0]                      
__________________________________________________________________________________________________
numerical_features (InputLayer) [(None, None, 5)]    0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, None, 4)      0           embedding[0][0]                  
_____________________________________________